# Average for small metric for LogME Imagenet combination

In [113]:
from itertools import combinations

import combination_analysis
import numpy as np
import pandas as pd
import utils
import wandb
import yaml
from constants import DATASET_NAMES, MAX_SIZES, METRIC_NAMES

In [114]:
# Load the data
api = wandb.Api()
datasets = list(MAX_SIZES.keys())
train_data = utils.get_data(api, "train", datasets)
metric_data = utils.get_data(api, "metrics", datasets)
data = metric_data.merge(train_data, on=["dataset_name", "model_name", "n_samples"])

KeyboardInterrupt: 

In [ ]:
# Read in imagenet validation accuracies
with open("../configs/scores_imagenet1k.yaml") as f:
    imagenet = yaml.safe_load(f)
imagenet = [
    {"model_name": key, "imagenet-validation": {"score": imagenet[key]}}
    for key in imagenet.keys()
]
imagenet = pd.DataFrame(imagenet)

In [ ]:
# Merge in imagenet val acc
data = data.merge(imagenet, on="model_name")

In [ ]:
# Scale test accuracy for regret
data["test_accuracy"] = data["test_accuracy"] * 100

In [ ]:
group_data = data.groupby(["dataset_name", "n_samples", "n_metric_samples"])

In [ ]:
corr = combination_analysis.get_correlations(
    group_data,
    "LogME_imagenet-validation",
    ["LogME", "imagenet-validation"],
    combination_analysis.zscore_scale,
)

In [ ]:
regret = combination_analysis.get_regrets(
    group_data,
    "LogME_imagenet-validation",
    ["LogME", "imagenet-validation"],
    1,
    combination_analysis.zscore_scale,
)

In [ ]:
small_corr = corr[corr["n_metric_samples"] == 1000]

In [ ]:
x = [v[0] for v in small_corr["LogME_imagenet-validation"]]

In [ ]:
def make_mean_plus_ci(x):
    m = np.mean(x)
    se = np.std(x) / np.sqrt(len(x))
    return {
        "mean": np.around(m, 2),
        "lower": np.around(m - 2 * se, 2),
        "upper": np.around(m + 2 * se, 2),
    }

In [ ]:
make_mean_plus_ci([v[0] for v in small_corr["LogME_imagenet-validation"].values])

{'mean': 0.58, 'lower': 0.54, 'upper': 0.62}

In [ ]:
small_regret = regret[regret["n_metric_samples"] == 1000]

In [ ]:
make_mean_plus_ci(small_regret["LogME_imagenet-validation"].values)

{'mean': 1.02, 'lower': 0.6, 'upper': 1.45}

Now to get these values for all of the combination

In [ ]:
metric_names = {metric: [metric] for metric in METRIC_NAMES}
for combo in combinations(METRIC_NAMES, 2):
    metric_names[combo[0] + "_" + combo[1]] = [combo[0], combo[1]]
for combo in combinations(METRIC_NAMES, 3):
    metric_names[combo[0] + "_" + combo[1] + "_" + combo[2]] = [
        combo[0],
        combo[1],
        combo[2],
    ]
metric_names[
    METRIC_NAMES[0]
    + "_"
    + METRIC_NAMES[1]
    + "_"
    + METRIC_NAMES[2]
    + "_"
    + METRIC_NAMES[3]
] = METRIC_NAMES

In [ ]:
data = data[data["n_metric_samples"] == 500]

In [ ]:
group_data = data.groupby(["dataset_name", "n_samples", "n_metric_samples"])

In [ ]:
corr_tables = []
for metric in metric_names.keys():
    corr_tables.append(
        combination_analysis.get_correlations(
            group_data, metric, metric_names[metric], combination_analysis.zscore_scale
        )
    )

In [ ]:
reget_tables = []
for metric in metric_names.keys():
    reget_tables.append(
        combination_analysis.get_regrets(
            group_data,
            metric,
            metric_names[metric],
            1,
            combination_analysis.zscore_scale,
        )
    )

In [ ]:
c_all = pd.concat(
    [
        combination_analysis.select_results(
            utils.select_sample_results,
            "n_samples",
            list(metric_names.keys()),
            *corr_tables,
            dataset_name=dataset_name
        )
        for dataset_name in DATASET_NAMES.values()
    ]
)

In [ ]:
r_all = pd.concat(
    [
        combination_analysis.select_results(
            utils.select_sample_results,
            "n_samples",
            list(metric_names.keys()),
            *reget_tables,
            dataset_name=dataset_name
        )
        for dataset_name in DATASET_NAMES.values()
    ]
)

In [ ]:
regret_tables = reget_tables

In [ ]:
regret_results = {
    list(metric_names.keys())[idx]: make_mean_plus_ci(
        tab[list(metric_names.keys())[idx]].values
    )
    for idx, tab in enumerate(reget_tables)
}

In [ ]:
corr_results = {
    list(metric_names.keys())[idx]: make_mean_plus_ci(
        [v[0] for v in tab[list(metric_names.keys())[idx]].values]
    )
    for idx, tab in enumerate(corr_tables)()
}

In [ ]:
results_df = pd.DataFrame(
    {
        "Metric": corr_results.keys(),
        "Correlation": corr_results.values(),
        "Regret": regret_results.values(),
    }
)

In [ ]:
results_df.to_csv("500_regret.csv")